# Mount Drive

In [77]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

#Navigate to /mydrive/yolov4
%cd /mydrive/MachineLearning/PittsburghTraffic/

# list the contents of /mydrive
!ls

/content/gdrive/MyDrive/MachineLearning
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
/content/gdrive/My Drive/MachineLearning/PittsburghTraffic
alcogisallegheny-county-owned-roads-centerlines  pgh_speed_viz.ipynb
pgh_pred.csv					 pgh_train.csv
pgh_segments.csv


# Read in files

In [48]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import numpy

df_seg = pd.read_csv ('pgh_segments.csv')
df_train = pd.read_csv ('pgh_train.csv')
df_pred = pd.read_csv ('pgh_pred.csv')
df_seg

,from.x,from.y,to.x,to.y
0,-79.866635,40.452687,-79.866439,40.452785
1,-79.867666,40.452143,-79.866635,40.452687
2,-79.867638,40.451004,-79.866897,40.450547
3,-79.867328,40.450173,-79.866971,40.450184
4,-79.867624,40.449684,-79.867066,40.449859
...,...,...,...,...
22576,-80.087996,40.453451,-80.087514,40.449512
22577,-80.084833,40.442184,-80.088807,40.446494
22578,-80.083976,40.442466,-80.088892,40.449447
22579,-80.087514,40.449512,-80.088892,40.449447


# Train

In [49]:
df_train

,from.x,from.y,to.x,to.y,speed,time,is.weekday
0,-80.010193,40.439438,-80.009608,40.439757,27.603437,8,1
1,-79.975452,40.435881,-79.966978,40.435817,53.779673,8,1
2,-79.926506,40.427954,-79.911553,40.425766,39.520281,8,1
3,-80.010193,40.439438,-80.000976,40.436302,24.878792,8,1
4,-80.007891,40.448433,-80.004527,40.449137,41.973019,8,1
...,...,...,...,...,...,...,...
565,-79.895381,40.425906,-79.892002,40.427876,51.251842,20,0
566,-79.895487,40.426092,-79.891258,40.427202,38.864311,20,0
567,-79.895487,40.426092,-79.891258,40.427202,46.044388,20,0
568,-79.926933,40.426905,-79.927907,40.427623,23.390307,20,0


In [52]:
# set coordinates from pd.Series to list
long1 = df_train["from.x"].to_list()
lat1 = df_train["from.y"].to_list()
long2 = df_train["to.x"].to_list()
lat2 = df_train["to.y"].to_list()

# append from/to x,y coordinates to new list alternating from previous to next step
long = []
lat = []

for i, j in zip(long1, long2):
  long += [i, j]

for i, j in zip(lat1, lat2):
  lat += [i, j]

points = list(zip(lat, long))

570.0

In [53]:
speed = df_train["speed"].to_list()


570

# Maps

In [56]:
avg_x = round(df_seg["from.x"].mean(), 2)
avg_y = round(df_seg["from.y"].mean(), 2)

#Define coordinates of where we want to center our map
pgh_center_coords = [avg_y, avg_x]

#Create the map
my_map = folium.Map(location = pgh_center_coords, zoom_start = 13)

# Add coordinates
folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(my_map)

#Display the map
my_map

## Heatmap

In [54]:
from folium.plugins import HeatMap

# Create a Map instance
m = folium.Map(location= pgh_center_coords, tiles = 'stamentoner', zoom_start=13, control_scale=True)

# Add heatmap to map instance
# Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(points).add_to(m)

# Show map
m

## Clustered point map

In [55]:
# Create a Map instance
m = folium.Map(location= pgh_center_coords, tiles = 'cartodbpositron', zoom_start=13, control_scale=True)

# Create a folium marker cluster
marker_cluster = MarkerCluster(points)

# Add marker cluster to map
marker_cluster.add_to(m)

# Show map
m

In [59]:
!pip install geopandas
!pip install fiona

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 6.3 MB 45.3 MB/s 
     |████████████████████████████████| 16.7 MB 45.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
import geopandas as gpd
from fiona.crs import from_epsg
from shapely.geometry import LineString, MultiLineString

# Filepaths
roads_fp = "alcogisallegheny-county-owned-roads-centerlines/CR.shp"

# Read files
roads = gpd.read_file(roads_fp)

# Re-project to WGS84, Folium requires all data to be in WGS84
roads['geometry'] = roads['geometry'].to_crs(epsg=4326)

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
roads['geoid'] = roads.index.astype(str)
roads['geoid']
# Select data
roads = roads[['geoid', 'geometry']]

# convert the dataframe to geojson
roads_jsontxt = roads.to_json()

In [85]:

def getLinesAsPointList(row, geom):
  """Returns a list of coordinate pair tuples for the line ('lat', 'lon') of a LineString geometry"""
  if isinstance(row[geom], MultiLineString):
    return []
  else:
      list_x = list(row[geom].coords.xy[0])
      list_y = list(row[geom].coords.xy[1])
      return list(zip(list_y, list_x))



In [101]:
# Calculate x and y coordinates of the line
roads['points_list'] = roads.apply(getLinesAsPointList, geom='geometry', axis=1)

# list of lat lon coordinate pair tuples
roadpoints = [a for a in roads['points_list'].tolist() if len(a) >=2 ]
roadpoints = []

for a in roads['points_list'].tolist():
  if len(a) >=2:
      roadpoints.append(a)
 

m = folium.Map(location= pgh_center_coords,  zoom_start=11, control_scale=True)

for road in roadpoints:
  folium.PolyLine(locations=road, color="blue", weight=4, opacity=1).add_to(m)

m

# Preprocess Map

In [28]:
avg_x = round(df_seg["from.x"].mean(), 2)
print(avg_x)
avg_y = round(df_seg["from.y"].mean(), 2)
print(avg_y)

-79.98
40.44


In [ ]:
#Define coordinates of where we want to center our map
pgh_center_coords = [avg_y, avg_x]

#Create the map
my_map = folium.Map(location = pgh_center_coords, zoom_start = 13)

#Display the map
my_map

# Preprocess Coordinates and Speed

In [46]:
# set coordinates from pd.Series to list
long1 = df_seg["from.x"].to_list()
lat1 = df_seg["from.y"].to_list()
long2 = df_seg["to.x"].to_list()
lat2 = df_seg["to.y"].to_list()

# append from/to x,y coordinates to new list alternating from previous to next step
beg_coord = []
end_coord = []

for i, j in zip(long1, lat1):
  beg_coord += [i, j]

for i, j in zip(long2, lat2):
  end_coord += [i, j]

coord = []
for i, j, in zip(beg_coord, end_coord):
  coord += [i, j]


print(beg_coord)

[[], -79.866634969173, 40.4526873862987, -79.8676656836759, 40.4521430103091, -79.8676375023984, 40.4510036209569, -79.8673283857495, 40.4501731937568, -79.8676243749329, 40.4496839126598, -79.8676017348935, 40.449463635719, -79.866634969173, 40.4526873862987, -79.8670655949572, 40.4498594968049, -79.8676375023984, 40.4510036209569, -79.8673283857495, 40.4501731937568, -79.8682998884663, 40.4511368368922, -79.8671921199804, 40.4540479258263, -79.8679395918417, 40.4554835598428, -79.8676017348935, 40.449463635719, -79.8688002606045, 40.4511183125721, -79.8676017348935, 40.449463635719, -79.8679395918417, 40.4554835598428, -79.8682998884663, 40.4511368368922, -79.8685409632123, 40.4518888813576, -79.8685681418969, 40.4555257556326, -79.8685681418969, 40.4555257556326, -79.8683043632203, 40.4494767583537, -79.8702556169569, 40.461170011447, -79.8692704283437, 40.4508391470073, -79.8700346500323, 40.4521824922129, -79.8688002606045, 40.4511183125721, -79.8690509666403, 40.4494595542594, -7

In [10]:
# set coordinates from pd.Series to list
long1 = df_seg["from.x"].to_list()
lat1 = df_seg["from.y"].to_list()
long2 = df_seg["to.x"].to_list()
lat2 = df_seg["to.y"].to_list()

# append from/to x,y coordinates to new list alternating from previous to next step
long = []
lat = []

for i, j in zip(long1, long2):
  long += [i, j]

for i, j in zip(lat1, lat2):
  lat += [i, j]

points = list(zip(lat, long))
new_points = points[:20]
new_points

[(40.4526873862987, -79.866634969173),
 (40.4527852669997, -79.8664394520467),
 (40.4521430103091, -79.8676656836759),
 (40.4526873862987, -79.866634969173),
 (40.4510036209569, -79.8676375023984),
 (40.4505472630247, -79.866897247775),
 (40.4501731937568, -79.8673283857495),
 (40.4501841147877, -79.8669711016535),
 (40.4496839126598, -79.8676243749329),
 (40.4498594968049, -79.8670655949572),
 (40.449463635719, -79.8676017348935),
 (40.4494511149475, -79.8671214609016),
 (40.4526873862987, -79.866634969173),
 (40.4540479258263, -79.8671921199804),
 (40.4498594968049, -79.8670655949572),
 (40.4501731937568, -79.8673283857495),
 (40.4510036209569, -79.8676375023984),
 (40.4520450701844, -79.8676032551455),
 (40.4501731937568, -79.8673283857495),
 (40.4510036209569, -79.8676375023984)]

In [12]:
folium.PolyLine(new_points, color="blue", weight=2.5, opacity=1).add_to(my_map)

my_map